# Best build by weighted relic effects

Given a list of desired relic effects to which we give a desirability value, find combinations of available Vessels and relics that maximizes the value of effects we want in our build.

In [23]:
import itertools
import yaml

from collections import Counter

from items import Vessel, Relic

In [ ]:
ironeye_vessels = [
    Vessel(("Yellow", "Green", "Green")),
    Vessel(("Red", "Blue", "Yellow")),
    Vessel(("Red", "Green", "White"))
]

wylder_vessels = [
    Vessel(("Red", "Red", "Blue")),
    Vessel(("Yellow", "Green", "Green"))
]

In [25]:
inventory = []

with open("relic_inventory.yml", "r") as f:
    db = yaml.safe_load(f)
    for entry in db:
        inventory.append(Relic(entry['color'], entry['effects']))

In [26]:
with open("effect_preferences.yml", "r") as f:
    effect_preferences = yaml.safe_load(f)

In [27]:
def score_relics_in_inventory(inventory, effect_preferences):
    for relic in inventory:
        relic.score = 0
        for effect in relic.effects:
            relic.score += effect_preferences.get(effect, 0)
    return inventory

In [28]:
score_relics_in_inventory(inventory, effect_preferences["Ironeye"])
score_relics_in_inventory(inventory, effect_preferences["Wylder"])

[Relic(color='Red', effects=['Lightning Pots in possession at start of expedition', 'Wraith Calling Bell in possession at start of expedition', "Changes compatible armament's skill to Flaming Strike at start of expedition"]),
 Relic(color='Red', effects=['[Revenant] Expend own HP to fully heal nearby allies when activating Art', 'HP Restoration upon Whip Attacks', 'Holy Attack Power Up +1']),
 Relic(color='Red', effects=['Character Skill Cooldown Reduction +2', 'Physical Attack Up +2']),
 Relic(color='Red', effects=['[Executor] While Character Skill is active, unlocking use of cursed sword restores HP', 'HP Restoration upon Thrusting Sword Attacks']),
 Relic(color='Red', effects=['HP Restoration upon Twinblade Attacks', 'Improved Fundamentalist Incantations', 'Treasure marked upon map']),
 Relic(color='Red', effects=['Character Skill Cooldown Reduction +1', 'Lightning Damage Negation Up', 'Improved Sleep Resistance']),
 Relic(color='Red', effects=['[Recluse] Collecting affinity residue

In [29]:
def get_relic_combinations(vessel, inventory):
    possible_relics = [
        [relic for relic in inventory if relic.color == slot_color or slot_color == "White"]
        for slot_color in vessel.slot_colors
    ]
    return [
        vessel for vessel in itertools.product(*possible_relics)
        if all(value == 1 for value in Counter(vessel).values())
    ]

In [30]:
def score_relic_combinations(relic_combinations):
    scores = {
        relic_combination: sum([relic.score for relic in relic_combination])
        for relic_combination in relic_combinations
    }
    return {
        k: v
        for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)
    }

In [31]:
possible_relic_combinations = get_relic_combinations(wylder_vessels[0], inventory)
score_relic_combinations(possible_relic_combinations)

{(Relic(color='Red', effects=['Character Skill Cooldown Reduction +2', 'Physical Attack Up +2']),
  Relic(color='Red', effects=['Improved Poise Near Totem Stela', 'Taking attacks improves attack power', 'Mind +2']),
  Relic(color='Blue', effects=['[Wylder] Art activation spreads fire in area', 'Changes compatible armament’s skill to Rain of Arrows at start of expedition', 'Character Skill Cooldown Reduction +3'])): 260,
 (Relic(color='Red', effects=['Improved Poise Near Totem Stela', 'Taking attacks improves attack power', 'Mind +2']),
  Relic(color='Red', effects=['Character Skill Cooldown Reduction +2', 'Physical Attack Up +2']),
  Relic(color='Blue', effects=['[Wylder] Art activation spreads fire in area', 'Changes compatible armament’s skill to Rain of Arrows at start of expedition', 'Character Skill Cooldown Reduction +3'])): 260,
 (Relic(color='Red', effects=['Character Skill Cooldown Reduction +2', 'Physical Attack Up +2']),
  Relic(color='Red', effects=['Stamina recovers with e